### 1. 최근접 이웃 협업 필터링


##### 협업 필터링 방식 중 하나인 최근접 이웃 협업 필터링에는 사용자 기반 최근접 이웃 방식과 아이템 기반 최근접 이웃 방식 두가지의 방법이 있습니다.

```

1-1) 사용자 기반 최근접 이웃 방식과 아이템 기반 최근접 이웃 방식에 대해 간단히 설명하시요.

```

```
답: 사용자 기반 최근접 이웃 방식은 사용자별 데이터가 판단 기준으로, 특정 사용자와 유사한 사용자를 유사도가 높은 순으로 n만큼 추출하여 해당 사용자들이 좋아하는 아이템을 추천하는 방식이다.
아이템 기반 최근접 이웃방식은 아이템별 데이터가 판단 기준으로, 특정 아이템과 유사한 이용패턴을 보이는 다른 아이템들을 유사도가 높은 순으로 n만큼 추출하여 다른 아이템을 이용해본 적 없는 사용자에게 해당 아이템을 추천하는 방식이다
```

```
1-2) 사용자 기반 최근접 이웃 방식보다 아이템 기반 최근접 이웃 방식이 더 정확도가 높고 많이 사용하는 이유를 설명해 보시오.
```

```
답: 사용자들이 비슷한 것을 좋아한다고 해서 사용자 간 취향이 비슷하다고 판단하기는 어려움. 매우 유명한 것은 대부분의 사람들이 좋아할 것이고, 좋아한다고 해도 평점을 매기지 않는 경우가 많기 때문에 판단하기에 어려움
```

### 2. 행렬분해

##### 2-1) 다음 행렬에 대해서 경사하강법을 이용한 행렬분해를 실습한 후 원본행렬과 비교해 보세요.

* 조건
```
K = 3
steps = 1000
learning_rate = 0.01
r_lambda = 0.01
```

In [1]:
import numpy as np

In [3]:
# 6X5 원본행렬

R = np.array([[1, 5, np.NaN, 2, np.NaN, np.NaN],
              [3, np.NaN, 1, 4, 3, np.NaN],
              [5, np.NaN, 2, 3, np.NaN, 1],
              [2, 5, np.NaN, 1, 2, 2],
              [3, 4, 2, np.NaN, 1, 1]
    
])

np.random.seed(1)
K = 3
steps = 1000
learning_rate = 0.01
r_lambda = 0.01

In [14]:
P = np.random.normal(scale = 1./K, size = (R.shape[0],K))
Q = np.random.normal(scale = 1./K, size = (R.shape[1],K))

# 0이 아닌 원소 추출
non_zeros = [(i,j,R[i,j]) for i in range(R.shape[0]) for j in range(R.shape[1]) if R[i,j]>0]

for step in range(steps):
    for i,j,r in non_zeros:
        eij = r-np.dot(P[i,:],Q[j,:].T)
        
        # update
        P[i,:] += learning_rate * (eij*Q[j,:]-r_lambda*P[i,:])
        Q[j,:] += learning_rate * (eij*P[i,:]-r_lambda*Q[i,:])

In [15]:
pred_matrix = np.dot(P, Q.T)
print('예측행렬: \n', np.round(pred_matrix, 3))

예측행렬: 
 [[ 1.     4.988  1.398  1.986  2.472  2.406]
 [ 2.998  0.538  0.986  3.986  2.984  1.052]
 [ 4.973  1.826  2.01   2.99   2.893  0.983]
 [ 1.992  4.992  1.786  1.005  1.983  1.975]
 [ 2.992  3.987  1.967 -0.407  1.001  1.021]]


In [16]:
print(R)

[[ 1.  5. nan  2. nan nan]
 [ 3. nan  1.  4.  3. nan]
 [ 5. nan  2.  3. nan  1.]
 [ 2.  5. nan  1.  2.  2.]
 [ 3.  4.  2. nan  1.  1.]]


##### 2-2) 다음 RMSE를 계산하는 함수를 사용하여 오차를 확인하세요.

In [18]:
# RMSE
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    
    full_pred_matrix = np.dot(P, Q.T)
    
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [19]:
get_rmse(R,P,Q,non_zeros)

0.015505959494228352

### 3. 콘텐츠 기반 필터링

##### Book.csv 파일을 로드하고 콘텐츠 기반 필터링 방법을 사용하여 ㅇㅇ와 관련있는 10개의 책을 추천하는 실습입니다.

In [20]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

##### 3-1) Book.csv 파일을 로드하고 결측값이 있는 행을 제거하고 Genres 행렬을 단어의 나열로 전처리 한 후 Countvectorizer을 사용하여 백터행렬을 생성하시오.
* 조건</br>
CountVectorizer(min_df=0, ngram_range=(1,2))

In [21]:
data = pd.read_csv('./Book.csv')
data.head()

,User-ID,Book-Rating,Book-Title,Book-Author,Publisher,genres
0,276725,3,Flesh Tones: A Novel,M. J. Rose,Ballantine Books,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
1,276726,5,Rites of Passage,Judith Rae,Heinle,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."
2,276727,4,The Notebook,Nicholas Sparks,Warner Books,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
3,276729,3,Help!: Level 1,Philip Prowse,Cambridge University Press,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam..."
4,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,Cambridge University Press,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."


In [46]:
# 결측값 행 제거
data_df = data.dropna(axis=0)

In [39]:
from ast import literal_eval

In [47]:
data_df["genres"]=data_df["genres"].apply(literal_eval)

In [49]:
data_df["genres"]=data_df["genres"].apply(lambda x: [i["name"] for i in x])

In [54]:
from sklearn.feature_extraction.text import CountVectorizer

In [61]:
data_df["genres_literal"]=data_df["genres"].apply(lambda x:" ".join(x))
count_vect = CountVectorizer(min_df=0, ngram_range = (1,2))
genre_mat = count_vect.fit_transform(data_df["genres_literal"])

##### 3-2) 생성된 백터 행렬으로 코사인 유사도를 계산하고 행렬로 변환한 뒤 유사도가 높은 순으로 정렬하시오.

In [59]:
from sklearn.metrics.pairwise import cosine_similarity

In [63]:
genre_sim = cosine_similarity(genre_mat, genre_mat)

In [76]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind)

[[  0  11  37 ... 221 222 369]
 [ 72   9   1 ...  55 323 369]
 [  2 268 351 ... 134 137 369]
 ...
 [184 367 288 ...  79 185 369]
 [368  34 141 ... 195 194   0]
 [369 311 254 ... 215 216   0]]


##### 3-3) 장르 유사도에 따라 영화를 추천하는 다음 함수의 '빈칸'을 채우고 'The Notebook'이라는 책을 좋아하는 사용자에게 보여줄 추천 결과를 출력하시오(10개).

In [80]:
data_df.columns

Index(['User-ID', 'Book-Rating', 'Book-Title', 'Book-Author', 'Publisher',
       'genres', 'genres_literal'],
      dtype='object')

In [81]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    title_movie = df[df['Book-Title'] == title_name]
    
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [83]:
similar_movies = find_sim_movie(data_df, genre_sim_sorted_ind, "The Notebook", 10)
similar_movies[['Book-Title', 'Book-Rating', 'Book-Author']]

[[  2 268 351  18 237  46 297  17  24  32]]


,Book-Title,Book-Rating,Book-Author
2,The Notebook,4,Nicholas Sparks
356,Oceano Mare,6,Alessandro Baricco
469,Contacto,6,Carl Sagan
21,How to Deal With Difficult People,7,Rick Brinkman
304,Verraten und verkauft. Roman.,3,Philippe Djian
62,Henry der Held.,10,Roddy Doyle
411,The Tower at Stony Wood,8,Patricia A. McKillip
20,Birdsong: A Novel of Love and War,8,Sebastian Faulks
29,The Pillars of the Earth,5,Ken Follett
38,Besiegt vom Sturm der Leidenschaft.,4,Patricia Potter
